In [ ]:
import numpy as np
import cv2

In [ ]:
import insightface

#### 1. Configure Insight Face Model

In [ ]:
from insightface.app import FaceAnalysis

In [7]:
# buffalo_l model
app_l = FaceAnalysis(name='buffalo_l', root='', providers=['CPUExecutionProvider'])
app_l.prepare(ctx_id=0, det_size=(640, 640))

download_path: models\buffalo_l


100%|██████████| 281857/281857 [08:37<00:00, 544.49KB/s] 


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [11]:
# Load buffalo sc model
app_sc = FaceAnalysis(name="buffalo_sc", root='')

app_sc.prepare(ctx_id=0, det_size=(640, 640))

download_path: models\buffalo_sc


100%|██████████| 14619/14619 [00:06<00:00, 2244.21KB/s]
d:\project\hackathon\.venv\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### 2. Load Image

In [8]:
img = cv2.imread('test_image_2.jpg')

cv2.imshow('image', img)
cv2.waitKey(0) # as soon as if i press esc key or x button, window will get close
cv2.destroyAllWindows()

#### 3. Apply image to insightface model and get results

In [9]:
results_l = app_l.get(img)

d:\project\hackathon\.venv\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [12]:
print(results_l)

[{'bbox': array([1020.8796,  182.7989, 1091.808 ,  277.7735], dtype=float32), 'kps': array([[1041.7598 ,  216.64877],
       [1075.6534 ,  216.33203],
       [1060.4402 ,  238.18265],
       [1041.0541 ,  247.29329],
       [1076.7659 ,  247.15   ]], dtype=float32), 'det_score': 0.92853606, 'landmark_3d_68': array([[ 1.01850623e+03,  2.15442139e+02,  4.76796875e+01],
       [ 1.02040723e+03,  2.25836395e+02,  4.71096344e+01],
       [ 1.02217462e+03,  2.36490601e+02,  4.56146545e+01],
       [ 1.02470142e+03,  2.46602386e+02,  4.22225838e+01],
       [ 1.02865857e+03,  2.55719101e+02,  3.47196503e+01],
       [ 1.03393372e+03,  2.62589355e+02,  2.85783463e+01],
       [ 1.03929419e+03,  2.68177124e+02,  2.31938076e+01],
       [ 1.04617151e+03,  2.73068390e+02,  1.63351288e+01],
       [ 1.05764209e+03,  2.76488068e+02,  1.32485857e+01],
       [ 1.06875305e+03,  2.73228516e+02,  1.82489662e+01],
       [ 1.07533643e+03,  2.67099915e+02,  2.72538280e+01],
       [ 1.08052539e+03,  2.60

In [13]:
len(results_l)

21

In [15]:
results_l[0].keys()

dict_keys(['bbox', 'kps', 'det_score', 'landmark_3d_68', 'pose', 'landmark_2d_106', 'gender', 'age', 'embedding'])

In [24]:
# draw the bounding box of each detected face
img_copy = img.copy()
gender_encode = ['Female', 'Male']
for res in results_l:
  x1, y1, x2, y2 = res['bbox'].astype(int)

  # draw rectangle
  cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 1)

  # key points
  kps = res['kps'].astype(int)
  for k1, k2 in kps:
    cv2.circle(img_copy, (k1, k2), 2, (0, 255, 255), -1)
  
  # detection score
  score = "score: {}%".format(int(res['det_score'] * 100))
  cv2.putText(img_copy, score, (x1, y1), cv2.FONT_HERSHEY_PLAIN, 1.2, (255, 0, 255))

  # age and gender
  gender = gender_encode[res['gender']]
  age = res['age']
  age_gender = f"{gender}::{age}"

  cv2.putText(img_copy, age_gender, (x1, y2 + 10), cv2.FONT_HERSHEY_PLAIN, 1.2, (255, 0, 255))



cv2.imshow('bbox', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()